# ecModel simulation tests

This notebook contains tests to see if simulating the ecModel with proteomics works as intended in the simulations service of DeCaF.

Note: using the requirements from [simulations](https://github.com/DD-DeCaF/simulations) (i.e. create a conda env and in it `pip install -r ./requirements/requirements.txt`)

Benjamín J. Sánchez, 2019-11-21

## 1. Loading Model and Data

In [1]:
import cobra
import os
import pandas as pd

#E. coli model:
#! [ ! -f "eciML1515.xml" ] && curl -O -L "https://raw.githubusercontent.com/BenjaSanchez/notebooks/master/e_coli_simulations/eciML1515.xml"
#ecModel = cobra.io.read_sbml_model("eciML1515.xml")
#os.remove("eciML1515.xml")

#Yeast model:
! [ ! -f "ecYeastGEM.xml" ] && curl -O -L "https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml"
ecModel = cobra.io.read_sbml_model("ecYeastGEM.xml")
os.remove("ecYeastGEM.xml")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 9434k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 19 9434k   19 1879k    0     0  1879k      0  0:00:05  0:00:01  0:00:04 1064k
 45 9434k   45 4270k    0     0  2135k      0  0:00:04  0:00:02  0:00:02 1543k
 68 9434k   68 6442k    0     0  2147k      0  0:00:04  0:00:03  0:00:01 1710k
 99 9434k   99 9365k    0     0  2341k      0  0:00:04  0:00:04 --:--:-- 1965k
100 9434k  100 9434k    0     0  2358k      0  0:00:04  0:00:04 --:--:-- 2444k


In [2]:
#proteomics data:
def reset_proteomics():
    #data = pd.read_csv("ecoli_data_heinemann2016.csv")  # E. coli
    data = pd.read_csv("yeast_ref_data.csv")  # yeast
    proteomics = []
    for key, value in data.iterrows():
        protein = {"identifier":value[0], "measurement":value[1], "uncertainty":0}
        proteomics.append(protein)
    return(proteomics)

proteomics = reset_proteomics()
print(proteomics[:10])
print(len(proteomics))

[{'identifier': 'A5Z2X5', 'measurement': 6.34e-05, 'uncertainty': 0}, {'identifier': 'O13297', 'measurement': 1.91e-07, 'uncertainty': 0}, {'identifier': 'O13516', 'measurement': 5.33e-06, 'uncertainty': 0}, {'identifier': 'O43137', 'measurement': 2.4e-06, 'uncertainty': 0}, {'identifier': 'P00128', 'measurement': 4.07e-05, 'uncertainty': 0}, {'identifier': 'P00358', 'measurement': 6.3e-06, 'uncertainty': 0}, {'identifier': 'P00359', 'measurement': 0.00018932200000000001, 'uncertainty': 0}, {'identifier': 'P00410', 'measurement': 1.22e-05, 'uncertainty': 0}, {'identifier': 'P00546', 'measurement': 4.09e-06, 'uncertainty': 0}, {'identifier': 'P00549', 'measurement': 3.68e-05, 'uncertainty': 0}]
1221


In [3]:
#uptake data:
uptake = []
glucose = {"identifier":"r_1714_REV", "measurement":1, "uncertainty":0}
uptake.append(glucose)

## 2. Functions

In [4]:
def flexibilize_proteomics(model, biomass_reaction, growth_rate, uptake_secretion_rates, proteomics):
    # set uptake rates in model:
    for uptake_rate in uptake_secretion_rates:
        try:
            metabolite = find_metabolite(
                model, uptake_rate["identifier"], uptake_rate["namespace"], "e"
            )
        except MetaboliteNotFound:
            continue
        else:
            exchange_reactions = metabolite.reactions.intersection(model.exchanges)
            if len(exchange_reactions) != 1:
                continue
            exchange_reaction = next(iter(exchange_reactions))
            lb, ub = bounds(uptake_rate["measurement"], uptake_rate["uncertainty"])
            exchange_reaction.bounds = (lb, ub)

    # reset growth rate in model:
    model.reactions.get_by_id(biomass_reaction).bounds = (0, 1000)

    # build a table with protein ids, met ids in model and values to constrain with:
    prot_df = pd.DataFrame()
    for protein in proteomics:
        protein_id = protein["identifier"]
        lb, ub = bounds(protein["measurement"], protein["uncertainty"])
        for met in model.metabolites:
            if protein_id in met.id:
                new_row = pd.DataFrame(data={"met_id": met.id, "value": ub}, index=[protein_id])
                prot_df = prot_df.append(new_row)

    # constrain the model with all proteins and optimize:
    limit_proteins(model, prot_df["value"])
    solution = model.optimize()
    new_growth_rate = solution.objective_value

    # while the model cannot grow to the desired level, remove the protein with
    # the highest shadow price:
    minimal_growth, ub = bounds(growth_rate["measurement"], growth_rate["uncertainty"])
    prots_to_remove = []
    while new_growth_rate < minimal_growth and not prot_df.empty:
        # get most influential protein in model:
        top_protein = top_shadow_prices(solution, list(prot_df["met_id"]))
        value = top_protein[top_protein.index[0]]
        top_protein = top_protein.index[0]
        top_protein = prot_df.index[prot_df["met_id"] == top_protein][0]
        print("working: " + top_protein + " (sp=" + str(value) + ") - mu = " + str(new_growth_rate))  
        
        # update data: append protein to list, remove from current dataframe and
        # increase the corresponding upper bound to +1000:
        prots_to_remove.append(top_protein)
        prot_df = prot_df.drop(labels=top_protein)
        limit_proteins(model, pd.Series(data=[1000], index=[top_protein]))

        # re-compute solution:
        solution = model.optimize()
        if solution.objective_value == new_growth_rate:     # the algorithm is stuck
            break
        new_growth_rate = solution.objective_value

    # update growth rate if optimization was not successful:
    if new_growth_rate < minimal_growth:
        if growth_rate["uncertainty"]:
            growth_rate["measurement"] = new_growth_rate + growth_rate["uncertainty"]
        else:
            growth_rate["measurement"] = new_growth_rate

    # update proteomics by removing flexibilized proteins:
    for protein in prots_to_remove:
        index = next((index for (index, d) in enumerate(proteomics) if d["identifier"] == protein), None)
        proteomics.pop(index)

    return growth_rate, proteomics


def limit_proteins(model, measurements):
    for protein_id, measure in measurements.items():
        try:
            rxn = model.reactions.get_by_id("prot_{}_exchange".format(protein_id))
        except KeyError:
            pass
        else:
            # update only upper_bound (as enzymes can be unsaturated):
            rxn.bounds = (0, measure)
    return


def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    return shadow_pr.sort_values()[:top]

def bounds(measurement, uncertainty):
    if uncertainty:
        return measurement - uncertainty, measurement + uncertainty
    else:
        return measurement, measurement

def find_metabolite(model, id, namespace, compartment):
    def query_fun(metabolite):
        if metabolite.compartment != compartment:
            return False

        result = _query_item(metabolite, id, namespace)
        if result:
            return result

        # If the original query fails, retry with the compartment id appended
        # to the identifier (a regular convenation with BiGG metabolites, but
        # may also be the case in other namespaces).
        return _query_item(metabolite, f"{id}_{compartment}", namespace)

    metabolites = model.metabolites.query(query_fun)
    if len(metabolites) == 0:
        raise MetaboliteNotFound(
            f"Could not find metabolite {id} or {id}_{compartment} in "
            f"namespace {namespace} and compartment {compartment} for model "
            f"{model.id}"
        )
    elif len(metabolites) > 1:
        raise IndexError(f"Expected single metabolite, found {metabolites}")
    else:
        return metabolites[0]

## 3. Testing

In [5]:
solution = ecModel.optimize()
print(solution)

<Solution 0.088 at 0x1b076321fc8>


In [6]:
measurements = pd.DataFrame()
for protein in proteomics:
    protein_id = protein["identifier"]
    lb, ub = bounds(protein["measurement"], protein["uncertainty"])
    for met in ecModel.metabolites:
        if protein_id in met.id:
            new_row = pd.DataFrame(data={"met_id": met.id, "value": ub}, index=[protein_id])
            measurements = measurements.append(new_row)
print(measurements.head())

                          met_id     value
P00128  prot_P00128__91__c__93__  0.000041
P00358  prot_P00358__91__c__93__  0.000006
P00359  prot_P00359__91__c__93__  0.000189
P00410  prot_P00410__91__c__93__  0.000012
P00549  prot_P00549__91__c__93__  0.000037


In [7]:
limit_proteins(ecModel, measurements["value"])
solution = ecModel.optimize()
print(solution.objective_value)

5.199145593769825e-05


In [8]:
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)

prot_P38604__91__c__93__   -189.05984
Name: shadow_prices, dtype: float64


In [9]:
#new_growth_rate, new_proteomics = flexibilize_proteomics(ecModel, "BIOMASS_Ec_iML1515_core_75p37M", {"measurement":0.8, "uncertainty":0.1}, [], proteomics)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, [], proteomics)

working: P38604 (sp=-189.0598397734483) - mu = 5.199145593769825e-05
working: P38972 (sp=-1761.0143524943996) - mu = 0.018666752136436607
working: P00931 (sp=-2074.8931245794574) - mu = 0.022201356433002306
working: Q12233 (sp=-4642.300091434081) - mu = 0.023065544305181932
working: P24521 (sp=-55317.86427789225) - mu = 0.01864212026164969
working: P07283 (sp=-4177.57190480627) - mu = 0.034757398247988174
working: Q06405 (sp=-4642.298754485921) - mu = 0.03710384909604921
working: Q12349 (sp=-4642.298694622616) - mu = 0.03854296121291276
working: P21306 (sp=-4642.298694622572) - mu = 0.043603066790051444
working: P25340 (sp=-67422.99117010784) - mu = 0.04146513956961635
working: P28777 (sp=-10112.224326534935) - mu = 0.05015663265961329
working: P10659 (sp=-15090.604451151283) - mu = 0.05839281321776375
working: P54839 (sp=-6752.045587652795) - mu = 0.0695460695528238


In [10]:
print(growth_rate)
print(len(proteomics))

{'measurement': 0.08, 'uncertainty': 0.01}
1208


## 4. Signs of Shadow Prices

Let's evaluate what happens if we take the positive shadow prices (if any) instead of the negative ones, by sorting in a decreasing order instead of an increasing order:

In [11]:
def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    return shadow_pr.sort_values(ascending=False)[:top]

proteomics = reset_proteomics()
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, [], proteomics)
print(growth_rate)
print(len(proteomics))

prot_P39979__91__c__93__    0.0
Name: shadow_prices, dtype: float64
working: P39979 (sp=0.0) - mu = 5.199145593769825e-05
{'measurement': 0.010051991455937699, 'uncertainty': 0.01}
1220


No positive shadow prices are detected, and hence the algorithm gets stuck. Finally, let's take the absolute value of all shadow prices and then rank in decreasing order, so that the sign doesn't matter:

In [12]:
def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    for met_id, sp in shadow_pr.items():
        shadow_pr[met_id] = abs(sp)
    return shadow_pr.sort_values(ascending=False)[:top]

proteomics = reset_proteomics()
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, [], proteomics)
print(growth_rate)
print(len(proteomics))

prot_P38604__91__c__93__    189.05984
Name: shadow_prices, dtype: float64
working: P38604 (sp=189.05983977344818) - mu = 5.199145593769825e-05
working: P38972 (sp=1760.993678094865) - mu = 0.01866653298779876
working: P24521 (sp=55317.864277892244) - mu = 0.01864212026164968
working: P00931 (sp=2074.8931245790336) - mu = 0.02220135643299561
working: Q12233 (sp=4642.298694622596) - mu = 0.02306553736504101
working: P07283 (sp=4177.57190480627) - mu = 0.03475739824798817
working: Q06405 (sp=4642.298814349269) - mu = 0.03710384957450237
working: Q12349 (sp=4642.298814349255) - mu = 0.03854296220695032
working: P21306 (sp=4642.298814349284) - mu = 0.04360306791459133
working: P25340 (sp=67422.99117010784) - mu = 0.04146513956961631
working: P28777 (sp=10112.22432653517) - mu = 0.05015663265961439
working: P10659 (sp=15090.604451151285) - mu = 0.0583928132177621
working: P54839 (sp=6752.045587652798) - mu = 0.06954606955282379
{'measurement': 0.08, 'uncertainty': 0.01}
1208


We see that we get the same results as with negative shadow prices, so no positive shadow prices are popping up in any of the flexibilization iterations.